# EDA

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [6]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [8]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [9]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [12]:
# tags_rdd.show(5)

Leave out timestamp...

In [13]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [14]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [15]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Similarity Based Recommender using Surprise

Begin by reimplementing the similarity based recommender.

In [16]:
# pip install surprise

In [17]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

mov_surprise = mov_surprise[mov_surprise['rating']!=0]

reader = Reader(rating_scale=(1,5))

In [18]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [19]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [26]:
print(mov_pDF.tail(5))

        userId                           title  rating
100831     610                    Split (2017)     4.0
100832     610   John Wick: Chapter Two (2017)     5.0
100833     610                  Get Out (2017)     5.0
100834     610                    Logan (2017)     5.0
100835     610  The Fate of the Furious (2017)     3.0


In [20]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
# algo = KNNBasic()
algo = SVD()

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

# print the first 5 predictions
predictions[:5]

[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=4.8771984835524345, details={'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.754638717718587, details={'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=4.815890037034838, details={'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=4.2307717583333995, details={'was_impossible': False})]

In [27]:
predictions[-5:]

[Prediction(uid=610, iid='United States of Leland, The (2003)', r_ui=3.501556983616962, est=3.3642857817040417, details={'was_impossible': False}),
 Prediction(uid=610, iid='A Home at the End of the World (2004)', r_ui=3.501556983616962, est=3.9601784135041065, details={'was_impossible': False}),
 Prediction(uid=610, iid='League of Ordinary Gentlemen, A (2004)', r_ui=3.501556983616962, est=3.8935970608876724, details={'was_impossible': False}),
 Prediction(uid=610, iid='I Think I Love My Wife (2007)', r_ui=3.501556983616962, est=3.2313508254879526, details={'was_impossible': False}),
 Prediction(uid=610, iid='Man of the Year (1995)', r_ui=3.501556983616962, est=3.4967239551622185, details={'was_impossible': False})]

In [22]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8721  0.8689  0.8769  0.8609  0.8676  0.8693  0.0053  
MAE (testset)     0.6697  0.6674  0.6752  0.6618  0.6667  0.6682  0.0044  
Fit time          3.00    2.77    2.96    2.80    2.67    2.84    0.12    
Test time         0.35    0.36    0.31    0.30    0.26    0.32    0.04    


{'test_rmse': array([0.87209268, 0.86891766, 0.87692285, 0.86086742, 0.86756916]),
 'test_mae': array([0.66973578, 0.66740167, 0.67518763, 0.66177259, 0.66667806]),
 'fit_time': (3.004497766494751,
  2.7695112228393555,
  2.956921100616455,
  2.801307201385498,
  2.6742496490478516),
 'test_time': (0.34892988204956055,
  0.36215710639953613,
  0.3130302429199219,
  0.3012969493865967,
  0.26192665100097656)}

In [23]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9571  0.9584  0.9560  0.9515  0.9598  0.9565  0.0028  
MAE (testset)     0.7297  0.7319  0.7291  0.7268  0.7315  0.7298  0.0018  
Fit time          0.14    0.15    0.15    0.16    0.15    0.15    0.01    
Test time         1.83    1.94    2.09    1.95    2.02    1.97    0.09    


{'test_rmse': array([0.95705719, 0.95842414, 0.95603549, 0.95145238, 0.95976063]),
 'test_mae': array([0.72970274, 0.73185797, 0.72910948, 0.72677405, 0.73152203]),
 'fit_time': (0.13529372215270996,
  0.15058279037475586,
  0.15323519706726074,
  0.15837955474853516,
  0.148087739944458),
 'test_time': (1.8297433853149414,
  1.943314790725708,
  2.0880744457244873,
  1.9499742984771729,
  2.0177345275878906)}

In [24]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9339  0.9180  0.9317  0.9264  0.9360  0.9292  0.0064  
MAE (testset)     0.7174  0.7088  0.7162  0.7111  0.7200  0.7147  0.0041  
Fit time          0.23    0.30    0.23    0.22    0.21    0.24    0.03    
Test time         2.85    2.27    2.34    2.44    2.26    2.43    0.22    


{'test_rmse': array([0.9338685 , 0.91802202, 0.93174491, 0.9264032 , 0.93599248]),
 'test_mae': array([0.71736192, 0.70875317, 0.71621146, 0.71114665, 0.71996945]),
 'fit_time': (0.22688841819763184,
  0.2972452640533447,
  0.2333359718322754,
  0.2213442325592041,
  0.21461009979248047),
 'test_time': (2.8454015254974365,
  2.267185688018799,
  2.3389337062835693,
  2.437272548675537,
  2.2563982009887695)}

# Comparing to Baseline Model

In [25]:
from src/baselines import 


mov_pDF.head()

SyntaxError: invalid syntax (<ipython-input-25-e4aed93defeb>, line 1)